<a href="https://colab.research.google.com/github/dsivov/NEBULA2/blob/master/notebooks/gdrive_tempalte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nebula on GDrive and Colab.


*   Please create nebula folders in "Colab Notebooks" on your GDrive before:


1.   nebula/data - Code and already downloaded models
2.   nebula/env - Package installation path
3.   nebula/models - Tranformer Models (Will be donloaded only once)




*   Install rclone on linux and run rclone config (Instructions: https://rclone.org/drive/)



*   Mount gdrive on linux for VSCode and git access: - 
`rclone mount gdrive:Colab\ Notebooks gdrive --vfs-cache-mode full --vfs-cache-max-size 64G --vfs-cache-max-age 120h --dir-cache-time 120h --vfs-read-chunk-size-limit 0 --poll-interval 1s`
Now you can edit code in editor (vscode) locally

*   All packages install will be pointed to nebula_env (line 17)



This part must be executed each time you connect to Colab kernel

In [1]:
import os, sys
from google.colab import drive
nebula_path = '/content/nebula'
isdir = os.path.isdir(nebula_path)
if not isdir:
  drive.mount('/content/mnt')
  os.symlink('/content/mnt/My Drive/Colab Notebooks/nebula', nebula_path)
else:
  print(os.listdir(nebula_path))
nebula_env = nebula_path + "/env"
nebula_models = nebula_path + "/models"
nebula_data = nebula_path + "/data"
nebula_tar = nebula_data + "/backup.tgz"
nebula_src = nebula_data + "/dbdata/NEBULA2"
sys.path.insert(0, nebula_src)
sys.path.insert(0, nebula_env)
os.environ['TRANSFORMERS_CACHE'] = nebula_models


['env', 'models', 'data']


Execute it just once, when setup GDrive
Download nebula code to Gdrive and untar it


In [ ]:

#!wget https://nebula-volume.s3.eu-central-1.amazonaws.com/backup.tgz -O $nebula_tar
#!cd $nebula_data; pwd; ls; tar zxvf backup.tgz; ls


Add here related packages, each subproject with own cell, 
don't forget to add --target=$nebula_env 
Comment out after first install - they are persists on GDrive

In [ ]:
#!pip install --target=$nebula_env python-arango==7.2.0
#!pip install --target=$nebula_env pymilvus==1.1.0

Example for MDMMT related packages

In [ ]:
# MDMMT packages, uncomment for first usage
#!pip install --target=$nebula_env  tf_slim 
#!pip install --target=$nebula_env ftfy
#!pip install --target=$nebula_env transformers

In [3]:
from nebula_api.nebula_enrichment_api import NRE_API

In [4]:

from nebula_api.mdmmt_api.mdmmt_api import MDMMT_API
mdmmt = MDMMT_API()


INFO:tensorflow:Restoring parameters from /content/nebula/data/dbdata/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predicti

Tests:


1.   DataBase connector
2.   MDMMT 
3.   Milvus API



In [ ]:
class DatabaseConnector():
  def __init__(self):
      self.arango_host = 'http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:8529'

  def connect_db(self, dbname):
      client = ArangoClient(hosts=self.arango_host)
      db = client.db(dbname, username='nebula', password='nebula')
      return (db)
    

In [ ]:
gdb = DatabaseConnector()
db = gdb.connect_db("nebula_development")


In [ ]:
def get_movie_url(movie_id, db):
  #nebula_movies = []
  url = ""
  query = 'FOR doc IN Movies FILTER doc._id == "{}" RETURN doc.url_path'.format(movie_id)
  cursor = db.aql.execute(query)
  for data in cursor:
      #print(data)
      url = data
  #print(url)
  return(url)

In [ ]:
get_movie_url('Movies/114208149', db)

'static/development/1035_The_Adjustment_Bureau_00_01_40_825-00_01_46_814.mp4'

In [5]:
vector = mdmmt.encode_text("test encoder")
print(vector)

tensor([ 0.0050, -0.0057,  0.0204,  ...,  0.0024,  0.0030, -0.0023],
       device='cuda:0')


Test Nebula main API

In [ ]:
nre = NRE_API()
nre.get_all_movies()

Test Milvus API

In [ ]:
from config_nebula.config import NEBULA_CONF
ncfg = NEBULA_CONF()
HOST, _PORT = ncfg.get_milvus_server()
print(HOST, " ", _PORT)
from nebula_api.milvus_api import MilvusAPI
milvus_events = MilvusAPI(
            'milvus', 'vcomet_mdmmt_embedded_event', 'nebula_visualcomet', 1536)
vector = vector.tolist()
milvus_events.search_vector(40, vector)